In [1]:
from src import *
import torch
from functools import partial
import matplotlib.pyplot as plt

from src import *
torch.autograd.set_detect_anomaly(True) 

### Load Model & Dataset

In [2]:
llama_size = "7b"
use_base_model = True

if not use_base_model:
    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-chat-hf"
else:
    model_name_or_path = f"meta-llama/Llama-2-{llama_size}-hf"

hf_model, hf_tokenizer = load_model_from_transformers(model_name_or_path)
model = from_hf_to_tlens(hf_model, hf_tokenizer, f"llama-{llama_size}", disable_grads=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model llama-7b into HookedTransformer
Moving model to device:  cuda


In [5]:
forget_dataset, forget_dataloader = create_dataset("harry_potter", model.tokenizer, batch_size=32, max_seq_length=1024)
retain_dataset, retain_dataloader = create_dataset("owt", model.tokenizer, batch_size=32, max_seq_length=512, num_examples=1000)

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
mask = NeuronLevelMask(model).cuda()

for param in model.parameters():
    param.requires_grad_(False)

train_mask(
    model=model,
    mask=mask,
    retain_dataloader=retain_dataloader,
    forget_dataloader=forget_dataloader,
    max_gpu_batch_size=4,
    learning_rate=1e-1
)

  1%|          | 1/100 [00:26<43:04, 26.10s/it]

Retain Loss: 2.373, Forget Loss: 0.442, Reg Loss: 0.000


  2%|▏         | 2/100 [00:51<42:10, 25.82s/it]

Retain Loss: 2.399, Forget Loss: 0.268, Reg Loss: 0.452


  3%|▎         | 3/100 [01:17<41:39, 25.77s/it]

...

Retain Loss: 2.412, Forget Loss: 0.007, Reg Loss: 0.050


 99%|█████████▉| 99/100 [42:26<00:25, 25.64s/it]

Retain Loss: 2.344, Forget Loss: 0.005, Reg Loss: 0.050


100%|██████████| 100/100 [42:52<00:00, 25.72s/it]

Retain Loss: 2.399, Forget Loss: 0.007, Reg Loss: 0.049


In [20]:
for i in range(32):
    print(torch.sum(mask.masks[f"mlp_{i}"] < 0.9))

tensor(57, device='cuda:0')
tensor(34, device='cuda:0')
tensor(22, device='cuda:0')
tensor(33, device='cuda:0')
tensor(53, device='cuda:0')
tensor(69, device='cuda:0')
tensor(68, device='cuda:0')
tensor(50, device='cuda:0')
tensor(62, device='cuda:0')
tensor(36, device='cuda:0')
tensor(37, device='cuda:0')
tensor(34, device='cuda:0')
tensor(35, device='cuda:0')
tensor(20, device='cuda:0')
tensor(17, device='cuda:0')
tensor(14, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(4, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(3, device='cuda:0')
tensor(5, device='cuda:0')
tensor(9, device='cuda:0')
tensor(8, device='cuda:0')
tensor(6, device='cuda:0')
